##### Copyright 2019 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TensorFlow 2 quickstart for beginners

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/quickstart/beginner"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

This short introduction uses [Keras](https://www.tensorflow.org/guide/keras/overview) to:

1. Build a neural network that classifies images.
2. Train this neural network.
3. And, finally, evaluate the accuracy of the model.

This is a [Google Colaboratory](https://colab.research.google.com/notebooks/welcome.ipynb) notebook file. Python programs are run directly in the browser—a great way to learn and use TensorFlow. To follow this tutorial, run the notebook in Google Colab by clicking the button at the top of this page.

1. In Colab, connect to a Python runtime: At the top-right of the menu bar, select *CONNECT*.
2. Run all the notebook code cells: Select *Runtime* > *Run all*.

Download and install TensorFlow 2. Import TensorFlow into your program:

Note: Upgrade `pip` to install the TensorFlow 2 package. See the [install guide](https://www.tensorflow.org/install) for details.

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf

Load and prepare the [MNIST dataset](http://yann.lecun.com/exdb/mnist/). Convert the samples from integers to floating-point numbers:

In [3]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

Build the `tf.keras.Sequential` model by stacking layers. Choose an optimizer and loss function for training:

In [4]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

For each example the model returns a vector of "[logits](https://developers.google.com/machine-learning/glossary#logits)" or "[log-odds](https://developers.google.com/machine-learning/glossary#log-odds)" scores, one for each class.

In [5]:
predictions = model(x_train[:1]).numpy()
predictions


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



array([[-0.08075342,  0.3592606 , -0.09926943, -0.37192813, -0.4155896 ,
        -0.00984061,  0.25198796, -0.5479506 ,  0.8382959 ,  0.3681331 ]],
      dtype=float32)

The `tf.nn.softmax` function converts these logits to "probabilities" for each class: 

In [6]:
tf.nn.softmax(predictions).numpy()

array([[0.08219814, 0.12763143, 0.08069015, 0.06143366, 0.05880909,
        0.08823868, 0.11464886, 0.05151822, 0.20606299, 0.12876886]],
      dtype=float32)

Note: It is possible to bake this `tf.nn.softmax` in as the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to
provide an exact and numerically stable loss calculation for all models when using a softmax output. 

The `losses.SparseCategoricalCrossentropy` loss takes a vector of logits and a `True` index and returns a scalar loss for each example.

In [7]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

This loss is equal to the negative log probability of the the true class:
It is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to `-tf.log(1/10) ~= 2.3`.

In [8]:
loss_fn(y_train[:1], predictions).numpy()

2.4277098

In [9]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

The `Model.fit` method adjusts the model parameters to minimize the loss: 

In [10]:
model.fit(x_train, y_train, epochs=5)

Train on 60000 samples
Epoch 1/5
60000/60000 [==============================] - 2s 31us/sample - loss: 0.2960 - accuracy: 0.9145
Epoch 2/5
60000/60000 [==============================] - 2s 26us/sample - loss: 0.1441 - accuracy: 0.9569
Epoch 3/5
60000/60000 [==============================] - 2s 26us/sample - loss: 0.1081 - accuracy: 0.9672
Epoch 4/5
60000/60000 [==============================] - 2s 26us/sample - loss: 0.0882 - accuracy: 0.9733
Epoch 5/5
60000/60000 [==============================] - 2s 28us/sample - loss: 0.0763 - accuracy: 0.9757


The `Model.evaluate` method checks the models performance, usually on a "[Validation-set](https://developers.google.com/machine-learning/glossary#validation-set)".

In [11]:
model.evaluate(x_test,  y_test, verbose=2)

10000/10000 - 0s - loss: 0.0816 - accuracy: 0.9742


[0.08158277370217257, 0.9742]

The image classifier is now trained to ~98% accuracy on this dataset. To learn more, read the [TensorFlow tutorials](https://www.tensorflow.org/tutorials/).

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [12]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [13]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[7.7166398e-09, 5.8433418e-09, 7.1743949e-07, 2.8301865e-05,
        3.1429453e-10, 9.1968879e-08, 5.9369874e-14, 9.9996471e-01,
        1.5198387e-07, 6.0814914e-06],
       [1.2705884e-09, 1.3419936e-05, 9.9990261e-01, 8.8201455e-07,
        2.6846725e-14, 3.8385613e-07, 1.8809518e-05, 2.2788715e-12,
        6.3880281e-05, 6.5717336e-12],
       [3.2520745e-07, 9.9597991e-01, 7.0668798e-04, 7.4753756e-05,
        6.0510072e-05, 2.4058021e-05, 1.2434453e-04, 1.9947977e-03,
        1.0264678e-03, 7.9798419e-06],
       [9.9940503e-01, 9.2046919e-11, 9.9279459e-06, 2.8155668e-08,
        1.7205701e-06, 1.5385403e-05, 5.6320179e-04, 1.5695833e-06,
        1.0151127e-07, 3.0370895e-06],
       [1.0974410e-06, 5.4689292e-10, 7.8729805e-05, 1.6808180e-07,
        9.9737024e-01, 5.3272443e-06, 1.2392499e-05, 1.4203851e-04,
        2.1038639e-05, 2.3689799e-03]], dtype=float32)>

In [30]:
import numpy as np
predictions = probability_model.predict(x_test)
predictions.shape

(10000, 10)

In [31]:
y_pred = np.argmax(predictions, axis = 1).astype(np.uint8)

In [32]:
y_true = y_test

In [33]:
from sklearn.metrics import accuracy_score
accuracy_score(y_true, y_pred)

0.9742